In [ ]:
! pip install langgraph langsmith langchain langchain_openai langchain_community python-dotenv

In [ ]:
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.checkpoint.postgres import PostgresSaver
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
model = os.getenv("OPENAI_MODEL")
db_uri = os.getenv("DB_URI")

model = ChatOpenAI(api_key=api_key, model=model)
DB_URI = db_uri

In [ ]:
with PostgresSaver.from_conn_string(DB_URI) as checkpointer:
    # checkpointer.setup()

    def call_model(state: MessagesState):
        response = model.invoke(state["messages"])
        return {"messages": response}

    builder = StateGraph(MessagesState)
    builder.add_node(call_model)
    builder.add_edge(START, "call_model")

    graph = builder.compile(checkpointer=checkpointer)

    config = {"configurable": {"thread_id": "2"}}

    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            break

        for chunk in graph.stream(
            {"messages": [{"role": "user", "content": user_input}]},
            config,
            stream_mode="values",
        ):
            chunk["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, what is my name?
================================== Ai Message ==================================

Hi! Your name is Amitav. How can I help you today?
================================ Human Message =================================

My wife's name is Priyanka
================================== Ai Message ==================================

That's lovely! Priyanka sounds like a beautiful name. How can I assist you today?
================================ Human Message =================================

What is my name and my wife's name?
================================== Ai Message ==================================

Your name is Amitav, and your wife's name is Priyanka. How can I assist you further?
================================ Human Message =================================

Can you book a flight to DElhi?
================================== Ai Message ==================================

I can't di